# Введение в MapReduce модель на Python


In [2]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [3]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.905589809636405),
 (1, 2.905589809636405),
 (2, 2.905589809636405),
 (3, 2.905589809636405),
 (4, 2.905589809636405)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0059671639991895065),
   (None, 0.07724245496172),
   (None, 0.08440804135613444),
   (None, 0.13575647907181598),
   (None, 0.14404826813474803),
   (None, 0.21204275967955666),
   (None, 0.21869633101751806),
   (None, 0.25055756276216923),
   (None, 0.28642389538931257),
   (None, 0.3834487515438496),
   (None, 0.3913614390023946),
   (None, 0.4041378102237341),
   (None, 0.41854626274930695),
   (None, 0.4704310153549396),
   (None, 0.4776995227348928),
   (None, 0.48992216726013693)]),
 (1,
  [(None, 0.5005353544023048),
   (None, 0.5135664686748047),
   (None, 0.53391984417089),
   (None, 0.5587932025401512),
   (None, 0.5673804905854288),
   (None, 0.6926646597910275),
   (None, 0.7237444251339501),
   (None, 0.7557883138083207),
   (None, 0.785709769245918),
   (None, 0.7937098630029404),
   (None, 0.7942646850708935),
   (None, 0.9160468126494941),
   (None, 0.9618068292060864),
   (None, 0.9820764489731459)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [4]:
import random
from functools import reduce

# Нахождение максимального значения в каждом элементе
def MAP(num_list):
    return max(num_list)

# Нахождение максимального значения среди всех элементов
def REDUCE(max1, max2):
    return max(max1, max2)

# Генерация случайных чисел
def RECORDREADER(min, max, count):
    return [random.randint(min, max) for _ in range(count)]

random_num = RECORDREADER(-100, 100, 15)
print(random_num)
mapped_values = map(MAP, [random_num])
res = reduce(REDUCE, mapped_values)
print("Max value:", res)

[-59, -74, 4, 40, 34, -16, 37, 96, -85, 66, -25, -22, -87, -100, 59]
Max value: 96


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [5]:
import random
from typing import Iterator, Tuple

# Создание пары (1, number)
def MAP(number):
    return (1, number)

# Вычисление среднего значения из итератора NamedTuple
def REDUCE(key, numbers: Iterator[Tuple[int, int]]):
    sum = 0
    count = 0
    for _, n in numbers:  # Пропускаем ключи
        sum += n
        count += 1
    yield sum / count

# Генерация случайных чисел
def RECORDREADER(min, max, count):
    return [random.randint(min, max) for _ in range(count)]

record = RECORDREADER(-100, 100, 100)
map_output = list(map(MAP, record))
print("MAP output:", map_output)
reduce_output = list(REDUCE(1, map_output))  # Группируем по фиктивному ключу 1
print("Reduce output:", reduce_output[0])

MAP output: [(1, 62), (1, 10), (1, -17), (1, -56), (1, 72), (1, 38), (1, -40), (1, -22), (1, -75), (1, 2), (1, 83), (1, 80), (1, 24), (1, -11), (1, -7), (1, 69), (1, -25), (1, -88), (1, 82), (1, -58), (1, -23), (1, 82), (1, 37), (1, 59), (1, 6), (1, 46), (1, 94), (1, 24), (1, -18), (1, 72), (1, -56), (1, -81), (1, -45), (1, 85), (1, -11), (1, -35), (1, -90), (1, -90), (1, -41), (1, -89), (1, -1), (1, -7), (1, -20), (1, 71), (1, -17), (1, -82), (1, 3), (1, 50), (1, -4), (1, 80), (1, 84), (1, -28), (1, -4), (1, -46), (1, -64), (1, 2), (1, -87), (1, -31), (1, 73), (1, 18), (1, -58), (1, -78), (1, -66), (1, 50), (1, -99), (1, -75), (1, -23), (1, 59), (1, 11), (1, 20), (1, 56), (1, -41), (1, 31), (1, -21), (1, 10), (1, 95), (1, -82), (1, -18), (1, 61), (1, 61), (1, -75), (1, 50), (1, 52), (1, -75), (1, -60), (1, 7), (1, 41), (1, 32), (1, 68), (1, -90), (1, -3), (1, -7), (1, 71), (1, -100), (1, 10), (1, -33), (1, -92), (1, 7), (1, -36), (1, -26)]
Reduce output: -3.27


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [20]:
import random
from typing import Iterator, NamedTuple, List, Tuple

# Создание пары (1, number)
def MAP(number):
    return (1, number)

# Вычисление среднего значения из итератора NamedTuple
def REDUCE(key, numbers: Iterator[int]):
    sum = 0
    count = 0
    for n in numbers:
        sum += n
        count += 1
    return sum / count

# Генерация случайных чисел
def RECORDREADER(min, max, count):
    return [random.randint(min, max) for _ in range(count)]

# Группировка значений по ключу с использованием сортировки
def group_by_key(iterable: List[Tuple[int, int]]):
    grouped_data = {}
    for key, value in sorted(iterable, key=lambda x: x[0]):
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

random_num = RECORDREADER(-100, 100, 100)
map_output = list(map(lambda x: MAP(x), random_num))
print("MAP output:", map_output)
shuffle_output = group_by_key(map_output)
print("Shuffle output:", shuffle_output)
reduce_output = list(map(lambda x: REDUCE(x[0], iter(x[1])), shuffle_output))
print("Reduce output:", reduce_output)

MAP output: [(1, 24), (1, 28), (1, -6), (1, -95), (1, -26), (1, -58), (1, 60), (1, 42), (1, 9), (1, -24), (1, 44), (1, -83), (1, 6), (1, 37), (1, 97), (1, 27), (1, 20), (1, -52), (1, -3), (1, 85), (1, -89), (1, 15), (1, 42), (1, -90), (1, 5), (1, -46), (1, 62), (1, -28), (1, -79), (1, -32), (1, 46), (1, 99), (1, 25), (1, -47), (1, -25), (1, 13), (1, 79), (1, -18), (1, 73), (1, -49), (1, -69), (1, -50), (1, 4), (1, -57), (1, 86), (1, 64), (1, 75), (1, -35), (1, 14), (1, -36), (1, 97), (1, -63), (1, -95), (1, 29), (1, 57), (1, 6), (1, -86), (1, -17), (1, 69), (1, 99), (1, 61), (1, 45), (1, -89), (1, 16), (1, -30), (1, 43), (1, -100), (1, 17), (1, -72), (1, 71), (1, -14), (1, -65), (1, -47), (1, -69), (1, 78), (1, -95), (1, 48), (1, -87), (1, -89), (1, -86), (1, -35), (1, -11), (1, 34), (1, 69), (1, 55), (1, -26), (1, -70), (1, -100), (1, -55), (1, 83), (1, -63), (1, -42), (1, 34), (1, -59), (1, 63), (1, -66), (1, 95), (1, -60), (1, 58), (1, 16)]
Shuffle output: [(1, [24, 28, -6, -95, -26

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [25]:
import numpy as np
from typing import Iterator

# "Выравнивание" вложенной структуры данных
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Группировка значений по ключу с использованием сортировки
def group_by_key(iterable):
    grouped_data = {}
    for key, value in sorted(iterable, key=lambda x: x[0]):
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

# Распределенная группировка данных по ключу
def group_by_key_distributed(map_partitions, partitioner):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[partitioner(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Определение разделителя (партиционера)
def partitioner(obj):
    global reducers
    return hash(obj) % reducers

# Функция для выполнения операций MapReduce
def map_reduce_distributed(input_format, map_func, reduce_func, partitioner=partitioner, combiner=None):
    # Применяем функцию MAP к каждой записи во входных данных, затем "выравниваем" результаты
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: map_func(*k1v1), record_reader)), input_format())

    # Если задан комбайнер (COMBINER), применяем его к данным после группировки по ключу
    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: combiner(*k2v2), group_by_key(map_partition))), map_partitions)

    # Выполняем распределенную группировку данных по ключу с использованием заданного партиционера
    reduce_partitions = group_by_key_distributed(map_partitions, partitioner)

    # Применяем функцию REDUCE к каждой группе данных после группировки и "выравниваем" результаты
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: reduce_func(*reduce_input_group),
                                                                                    reduce_partition[1]))), reduce_partitions)

    # Вычисляем общее количество отправленных по сети пар ключ-значение
    total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])

    # Выводим информацию о количестве отправленных данных и возвращаем результат операций MapReduce
    print(f"{total_values_sent} key-value pairs were sent over a network.")
    return reduce_outputs

# Задание текстовых документов
doc1 = """
the sun is shining brightly
the sun is shining brightly
the sun is shining brightly"""
doc2 = """
what a beautiful day
what a beautiful day"""
doc3 = """
the dog is very happy"""
text_documents = [doc1, doc2, doc3, doc1, doc2, doc3]

# Задание количества задач Map и Reduce
num_mappers = 3
num_reducers = 2

def INPUTFORMAT():
    # Форматирование входных данных для MapReduce
    global maps

    # Функция RECORDREADER принимает на вход раздел (split) данных и генерирует ключ-значение для каждой строки документа
    def RECORDREADER(split):
      # Перебираем каждый документ в разделе, присваивая ему уникальный идентификатор docid
        for (docid, document) in enumerate(split):
          # Для каждой строки в документе создаем уникальный идентификатор lineid и разбиваем строку по символу новой строки '\n'
            for (lineid, line) in enumerate(document.split('\n')):
               # Генерируем ключ в формате "docid:lineid" и значение - строку из документа
                yield ("{}:{}".format(docid, lineid), line)
    # Вычисляем размер раздела данных split_size как округленное вверх
    #значение от деления общего числа документов на количество задач MapReduce
    split_size = int(np.ceil(len(text_documents) / maps))
    # Цикл по документам с шагом split_size
    for i in range(0, len(text_documents), split_size):
      # Генерируем данные для каждого раздела данных путем вызова функции RECORDREADER с подмножеством документов
        yield RECORDREADER(text_documents[i:i + split_size])

def MAP(docId: str, line: str):
    # Функция Map для преобразования данных
    for word in line.split(" "):
        yield (word, word)

def REDUCE(key: str, value: Iterator[str]):
    # Функция Reduce для агрегации данных
    yield key

# Выполнение MapReduce операций с заданными функциями MAP и REDUCE
partitioned_output = map_reduce_distributed(INPUTFORMAT, MAP, REDUCE, combiner=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

62 key-value pairs were sent over a network.


[(0,
  ['',
   'a',
   'beautiful',
   'brightly',
   'day',
   'dog',
   'happy',
   'is',
   'shining',
   'very',
   'what']),
 (1, ['sun', 'the'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [26]:
from collections import defaultdict
import random

# Функция для реализации MAP операции
def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C - выбираем только кортежи, удовлетворяющие условию
            mapped_result[t].append(t)
    return mapped_result.items()

# Функция для реализации REDUCE операции
def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)  # Выводим промежуточный результат
    for values in mapped_items:
        for value in values:
            reduced_result.append(value)  # Функция идентичности: возвращаем то же значение, что получили на вход
    return reduced_result

# Предикат для фильтрации кортежей с четным первым элементом
def C(t):
    return t[0] % 2 == 0

# Генерация случайных записей (кортежей формата (x, y, z))
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

# Создание набора записей
record = RECORDREADER(5)
print(record)

# Разбиение записей на равные части
part_count = 5
record_partitioned = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitioned)

# Применение операции MAP и последующей REDUCE к разделенным записям
print(REDUCE(list(map(lambda x: MAP(x), record_partitioned))))

[(47, 54, 55), (41, 23, 87), (2, 8, 79), (84, 13, 23), (88, 70, 44)]
[[(47, 54, 55), (41, 23, 87), (2, 8, 79), (84, 13, 23), (88, 70, 44)]]
[dict_items([((2, 8, 79), [(2, 8, 79)]), ((84, 13, 23), [(84, 13, 23)]), ((88, 70, 44), [(88, 70, 44)])])]
[((2, 8, 79), [(2, 8, 79)]), ((84, 13, 23), [(84, 13, 23)]), ((88, 70, 44), [(88, 70, 44)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [27]:
S = set([25, 90, 69])  # Создаем множество S

def MAP(t):
    res_list = []
    for el in t:  # Перебираем элементы входного кортежа t
        if el in S:  # Проверяем, есть ли элемент el в множестве S
            res_list.append(el)  # Добавляем элемент el в результирующий список
    res = tuple(res_list)  # Преобразуем список в кортеж
    return (res, res)  # Возвращаем кортеж из результирующего списка дважды

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(5)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(100)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список

print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем функцию REDUCE к результату группировки
print("Reduce output:", reduce_output)

[(55, 76, 54), (80, 70, 77), (20, 84, 3), (46, 17, 59), (32, 0, 77)]
MAP output: [((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [28]:
def MAP(t):
    return (t, t)  # Функция MAP возвращает кортеж из входного элемента и этого же элемента

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Функция REDUCE возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(5)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(100)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем функцию REDUCE к результату группировки
print("Reduce output:", reduce_output)

[(25, 95, 80), (21, 1, 23), (86, 61, 98), (57, 23, 3), (25, 32, 76)]
MAP output: [((7, 22, 80), (7, 22, 80)), ((54, 26, 51), (54, 26, 51)), ((54, 84, 100), (54, 84, 100)), ((44, 96, 65), (44, 96, 65)), ((28, 49, 57), (28, 49, 57)), ((18, 4, 47), (18, 4, 47)), ((84, 64, 31), (84, 64, 31)), ((13, 73, 36), (13, 73, 36)), ((6, 74, 38), (6, 74, 38)), ((9, 68, 22), (9, 68, 22)), ((27, 95, 43), (27, 95, 43)), ((60, 81, 8), (60, 81, 8)), ((64, 49, 22), (64, 49, 22)), ((32, 23, 96), (32, 23, 96)), ((5, 58, 65), (5, 58, 65)), ((65, 27, 41), (65, 27, 41)), ((93, 55, 26), (93, 55, 26)), ((62, 56, 100), (62, 56, 100)), ((87, 99, 0), (87, 99, 0)), ((30, 49, 28), (30, 49, 28)), ((59, 43, 74), (59, 43, 74)), ((71, 51, 79), (71, 51, 79)), ((59, 60, 65), (59, 60, 65)), ((85, 88, 84), (85, 88, 84)), ((39, 14, 100), (39, 14, 100)), ((40, 81, 26), (40, 81, 26)), ((32, 81, 89), (32, 81, 89)), ((31, 63, 31), (31, 63, 31)), ((79, 72, 13), (79, 72, 13)), ((2, 76, 92), (2, 76, 92)), ((90, 61, 55), (90, 61, 55))

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [30]:
def MAP(t):
    return (t, t)  # Функция MAP возвращает кортеж из входного элемента и этого же элемента

def REDUCE(key, values: Iterator[NamedTuple]):
    if len(values) == 2:  # Если количество значений равно 2
        return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(100)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(10)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список
print("Shuffle output:", shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]  # Применяем функцию REDUCE к результату группировки
print("Reduce output:", reduce_output)

[(1, 2), (1, 1), (0, 0), (1, 1), (1, 3), (0, 1), (0, 2), (0, 2), (3, 3), (2, 3), (1, 0), (0, 2), (3, 1), (1, 1), (3, 1), (3, 0), (0, 3), (3, 1), (1, 2), (1, 2), (0, 0), (3, 1), (2, 0), (2, 1), (1, 3), (1, 2), (3, 3), (0, 2), (0, 2), (3, 1), (1, 1), (1, 0), (3, 0), (1, 2), (1, 3), (0, 3), (3, 1), (3, 1), (0, 1), (1, 2), (0, 2), (1, 3), (0, 3), (0, 2), (3, 2), (2, 0), (0, 3), (1, 3), (2, 3), (2, 3), (3, 3), (1, 0), (3, 0), (0, 0), (2, 1), (1, 3), (0, 1), (3, 3), (1, 1), (1, 3), (3, 2), (2, 1), (1, 1), (0, 1), (3, 2), (2, 1), (0, 0), (2, 3), (0, 0), (2, 3), (2, 1), (1, 2), (3, 1), (3, 3), (3, 3), (3, 1), (3, 2), (0, 1), (2, 1), (1, 2), (3, 0), (0, 2), (3, 3), (1, 0), (2, 0), (1, 3), (1, 1), (2, 3), (0, 0), (3, 1), (2, 2), (1, 1), (2, 1), (0, 3), (0, 2), (0, 0), (3, 2), (3, 0), (0, 0), (0, 0)]
MAP output: [((3, 2), (3, 2)), ((1, 1), (1, 1)), ((1, 3), (1, 3)), ((1, 0), (1, 0)), ((1, 2), (1, 2)), ((3, 2), (3, 2)), ((0, 0), (0, 0)), ((3, 0), (3, 0)), ((3, 2), (3, 2)), ((2, 0), (2, 0)), ((3, 3

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [31]:
rels = [1, 2]  # Инициализация массива rels с элементами 1 и 2

class Tuple:  # Определение класса Tuple для хранения данных и id представления
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])  # Генерация случайного кортежа данных
    rel_id = rels[random.randint(0, len(rels) - 1)]  # Выбор случайного id представления из массива rels
    return Tuple(data, rel_id)  # Возвращает экземпляр класса Tuple с данными и id представления

def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]  # Генерация списка случайных кортежей данных

def MAP(t: Tuple):
    return (t.data, t.rel_id)  # Функция MAP возвращает кортеж из данных и id представления

def REDUCE(key, values: Iterator[NamedTuple]):
    if values == [rels[0]]:  # Если значение соответствует первому элементу в массиве rels
        return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]  # Группировка значений по ключу k2
    return t.items()  # Возвращает список кортежей (ключ, список значений)

record = RECORDREADER(100)  # Генерация списка случайных кортежей данных

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу из RECORDREADER(100)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка значений по ключу
shuffle_output = list(shuffle_output)  # Преобразование результата в список
print("Shuffle output:", shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]  # Применение функции REDUCE к результату группировки
print("Reduce output:", reduce_output)

MAP output: [(((2, 1), (1, 0), (0, 1)), 2), (((1, 3), (1, 0), (3, 3)), 2), (((0, 0), (3, 0), (2, 0)), 2), (((3, 2), (1, 1), (3, 3)), 1), (((3, 3), (3, 1), (2, 0)), 1), (((2, 2), (0, 1), (2, 1)), 2), (((2, 1), (1, 0), (1, 0)), 2), (((2, 1), (2, 0), (3, 2)), 2), (((1, 3), (3, 3), (1, 2)), 1), (((0, 1), (3, 1), (1, 0)), 2), (((1, 0), (2, 0), (2, 3)), 2), (((3, 0), (2, 0), (3, 1)), 1), (((0, 3), (1, 2), (1, 3)), 1), (((2, 0), (0, 3), (1, 0)), 2), (((3, 0), (3, 2), (0, 1)), 1), (((1, 3), (0, 0), (0, 3)), 1), (((3, 3), (0, 3), (0, 1)), 2), (((2, 3), (0, 3), (0, 3)), 2), (((3, 2), (2, 2), (3, 2)), 2), (((0, 1), (1, 0), (2, 0)), 1), (((2, 1), (2, 1), (3, 1)), 2), (((3, 1), (0, 3), (1, 1)), 2), (((1, 1), (3, 0), (1, 0)), 1), (((3, 0), (1, 2), (1, 1)), 2), (((2, 3), (0, 1), (3, 2)), 1), (((0, 2), (1, 1), (0, 1)), 1), (((3, 1), (3, 3), (3, 1)), 1), (((3, 2), (0, 0), (1, 2)), 1), (((3, 2), (2, 2), (2, 2)), 1), (((0, 0), (0, 1), (1, 2)), 2), (((2, 1), (0, 2), (3, 1)), 1), (((3, 2), (0, 1), (2, 1)),

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [32]:
rels = [1, 2]  # Инициализация списка идентификаторов связей

class Tuple:  # Определение класса кортежа, хранящего данные и идентификатор связи из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():  # Функция для генерации случайного кортежа с данными и идентификатором связи
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):  # Функция для преобразования данных кортежа в соответствии с условием
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для объединения данных по ключу
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):  # Функция для группировки данных по ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)  # Создание списка случайных кортежей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print("Reduce output:", reduce_output)

MAP output: [(1, (1, 1)), (1, (2, 3)), (1, (2, 1)), (0, (2, 3)), (1, (2, 1)), (2, (1, 0)), (2, (2, 1)), (3, (2, 3)), (0, (1, 1)), (2, (1, 3)), (0, (2, 0)), (3, (1, 0)), (3, (1, 3)), (2, (1, 1)), (2, (2, 0)), (3, (1, 2)), (2, (2, 0)), (0, (2, 0)), (2, (1, 2)), (2, (2, 0)), (1, (1, 0)), (2, (2, 2)), (3, (2, 3)), (2, (2, 3)), (0, (2, 2)), (1, (2, 2)), (0, (2, 0)), (3, (1, 3)), (3, (2, 2)), (1, (1, 2)), (1, (1, 3)), (1, (2, 1)), (2, (2, 2)), (0, (1, 3)), (1, (2, 3)), (1, (2, 3)), (0, (2, 1)), (3, (1, 1)), (1, (2, 0)), (1, (1, 2)), (2, (2, 0)), (2, (1, 3)), (3, (1, 0)), (3, (1, 3)), (3, (2, 2)), (0, (2, 2)), (0, (2, 1)), (0, (2, 0)), (1, (2, 0)), (0, (1, 2)), (3, (2, 3)), (0, (2, 0)), (1, (1, 3)), (0, (2, 3)), (3, (2, 3)), (1, (1, 0)), (2, (1, 0)), (3, (1, 2)), (1, (2, 2)), (1, (2, 2)), (3, (2, 2)), (1, (1, 3)), (3, (2, 2)), (3, (1, 0)), (1, (2, 1)), (1, (1, 0)), (2, (2, 2)), (0, (1, 0)), (1, (2, 2)), (0, (1, 1)), (1, (2, 1)), (3, (2, 0)), (0, (1, 0)), (0, (1, 3)), (3, (2, 3)), (0, (1, 2)),

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [33]:
def get_random_tuple():  # Функция для генерации случайного кортежа с тремя случайными значениями от 0 до 3
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):  # Функция для преобразования входного кортежа, возвращающая первые два элемента кортежа
  return (t[0], t[1])

def tetta(values):  # Функция для вычисления суммы значений в переданном списке
  return sum(values)

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для вычисления суммы значений в переданных значениях и возврата пары (ключ, сумма)
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):  # Функция для группировки данных по ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)  # Создание списка случайных кортежей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print("Reduce output:", reduce_output)

MAP output: [(2, 2), (0, 0), (3, 3), (1, 0), (3, 2), (2, 3), (2, 3), (0, 1), (3, 3), (2, 3), (3, 0), (0, 2), (0, 2), (0, 2), (1, 3), (1, 1), (1, 0), (1, 0), (3, 3), (3, 0), (3, 2), (2, 0), (2, 0), (2, 1), (3, 2), (1, 1), (3, 3), (3, 2), (1, 1), (3, 2), (3, 2), (1, 1), (2, 1), (0, 2), (1, 1), (3, 1), (1, 1), (1, 3), (0, 1), (3, 3), (1, 1), (3, 3), (2, 1), (0, 2), (2, 3), (0, 1), (0, 1), (0, 1), (3, 0), (2, 0), (0, 2), (0, 0), (0, 3), (3, 2), (3, 0), (2, 3), (3, 0), (2, 2), (1, 2), (2, 2), (1, 2), (3, 2), (3, 1), (1, 1), (3, 2), (3, 1), (3, 1), (1, 3), (1, 3), (0, 0), (1, 2), (0, 2), (2, 2), (3, 1), (3, 2), (1, 0), (0, 3), (2, 0), (3, 1), (0, 0), (2, 3), (2, 2), (3, 0), (2, 0), (1, 2), (1, 1), (1, 3), (3, 0), (3, 3), (1, 2), (1, 2), (0, 0), (3, 2), (0, 1), (3, 1), (3, 0), (2, 3), (1, 1), (1, 2), (1, 2)]
Shuffle output: [(2, [2, 3, 3, 3, 0, 0, 1, 1, 1, 3, 0, 3, 2, 2, 2, 0, 3, 2, 0, 3]), (0, [0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 0, 3, 0, 2, 3, 0, 0, 1]), (3, [3, 2, 3, 0, 3, 0, 2, 2, 3, 2, 2

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [36]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [37]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [38]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [39]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [40]:
import numpy as np  # Импорт библиотеки numpy под псевдонимом np
I = 2  # Задание значения переменной I равным 2
J = 3  # Задание значения переменной J равным 3
K = 4*10  # Задание значения переменной K равным 40
small_mat = np.random.rand(I,J)  # Создание случайной матрицы размером IxJ
big_mat = np.random.rand(J,K)  # Создание случайной матрицы размером JxK
reference_solution = np.matmul(small_mat, big_mat)  # Вычисление матричного произведения small_mat и big_mat

def RECORDREADER():  # Определение функции RECORDREADER без аргументов
  for i in range(small_mat.shape[0]):  # Цикл по строкам матрицы small_mat
    for j in range(small_mat.shape[1]):  # Цикл по столбцам матрицы small_mat
      yield ((0, i, j), small_mat[i,j])  # Генерация кортежа для первой матрицы

  for j in range(big_mat.shape[0]):  # Цикл по строкам матрицы big_mat
    for k in range(big_mat.shape[1]):  # Цикл по столбцам матрицы big_mat
      yield ((1, j, k), big_mat[j,k])  # Генерация кортежа для второй матрицы

def MAP_JOIN(k1, v1):  # Определение функции MAP_JOIN с ключем k1 и значением v1
  (mat_num, i, j) = k1  # Распаковка ключа
  w = v1  # Присвоение значения из значения
  if mat_num == 0:  # Если номер матрицы равен 0
    yield (j, (mat_num, i, w))  # Генерация значения для первой матрицы
  else:  # Иначе
    yield (i, (mat_num, j, w))  # Генерация значения для второй матрицы

def REDUCE_JOIN(key, values):  # Определение функции REDUCE_JOIN с ключем key и значениями values
  from_first_mat = [v for v in values if v[0] == 0]  # Фильтрация значений для первой матрицы
  from_second_mat = [v for v in values if v[0] == 1]  # Фильтрация значений для второй матрицы
  for f in from_first_mat:  # Цикл по элементам из первой матрицы
    for s in from_second_mat:  # Цикл по элементам из второй матрицы
      yield ((f[1], s[1]), f[2] * s[2])  # Генерация умноженных значений


def MAP_MUL(k1, v1):  # Определение функции MAP_MUL с ключем k1 и значением v1
  (i, k) = k1  # Распаковка ключа
  yield ((i, k), v1)  # Генерация кортежа с ключом (i, k) и значением v1

def REDUCE_MUL(key, values):  # Определение функции REDUCE_MUL с ключем key и значениями values
  res_el_value = 0  # Инициализация переменной для хранения результата
  for v in values:  # Цикл по значениям
    res_el_value += v  # Суммирование значений
  yield (key, res_el_value)  # Генерация кортежа с ключом key и суммой значений

def GET_JOINED():  # Определение функции GET_JOINED без аргументов
  for j in joined:  # Цикл по элементам из joined
    yield j  # Генерация элемента


joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)  # Применение MapReduce к функциям RECORDREADER, MAP_JOIN и REDUCE_JOIN
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)  # Применение MapReduce к функциям GET_JOINED, MAP_MUL и REDUCE_MUL
np.allclose(reference_solution, asmatrix(solution))  # Проверка на равенство решения с эталонным значением и вывод результата (True/False)


True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [41]:
# Определение размеров матриц
I = 2
J = 3
K = 4*10

# Генерация случайных матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  # Разделение данных на части и применение MAP функции
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  # Применение COMBINER (если есть)
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  # Распределенная группировка данных по ключу (shuffle)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  # Применение REDUCE функции
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Преобразование результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

# Проверка результата на соответствие эталонному решению
np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.5663409637196033), ((0, 1), 0.4960331222371311), ((0, 2), 0.535643439447724), ((0, 3), 0.06412624665524652), ((0, 4), 0.5660837854156591), ((0, 5), 0.44491173357994396), ((0, 6), 0.648699488243651), ((0, 7), 0.31684982286509655), ((0, 8), 0.06630933204213373), ((0, 9), 0.6255687198037236), ((0, 10), 0.4814865511568245), ((0, 11), 0.6615596720287241), ((0, 12), 0.1419321261962329), ((0, 13), 0.5699576400027143), ((0, 14), 0.6712597077906054), ((0, 15), 0.4375332453068215), ((0, 16), 0.28204612251769706), ((0, 17), 0.5416837368394638), ((0, 18), 0.6118052451608603), ((0, 19), 0.2597127202645067), ((0, 20), 0.53929699633732), ((0, 21), 0.33351407260083943), ((0, 22), 0.6274267862816264), ((0, 23), 0.7886989076324225), ((0, 24), 0.6338871420449145), ((0, 25), 0.7452674775338265), ((0, 26), 0.7868936584423095), ((0, 27), 0.17032912723329138), ((0, 28), 0.4181668191576021), ((0, 29), 0.6400402567533942), ((0, 30), 0.5582405896971

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [42]:
# Определение размеров матриц
I = 2  # Количество строк первой матрицы и строк в итоговой матрице
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4*10  # Количество столбцов второй матрицы и столбцов в итоговой матрице

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Генерация первой матрицы
big_mat = np.random.rand(J, K)  # Генерация второй матрицы

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.19625370466672884), ((0, 1), 0.5161318546333956), ((0, 2), 0.2767143986332856), ((0, 3), 0.5363602088230293), ((0, 4), 0.2597487999447544), ((0, 5), 0.059498429890318935), ((0, 6), 0.25151001272451734), ((0, 7), 0.06114272641900986), ((0, 8), 0.49800547703007747), ((0, 9), 0.18317243254543147), ((0, 10), 0.3136806021143372), ((0, 11), 0.3788899596706355), ((0, 12), 0.40642157354152897), ((0, 13), 0.5008893848817535), ((0, 14), 0.5428256978767624), ((0, 15), 0.29182206999924154), ((0, 16), 0.03004230240527001), ((0, 17), 0.196036178842705), ((0, 18), 0.25130221632456884), ((0, 19), 0.16486943577325547), ((0, 20), 0.23264934162837017), ((0, 21), 0.26912711194762345), ((0, 22), 0.19134856151733204), ((0, 23), 0.30862051424780745), ((0, 24), 0.2524409732336972), ((0, 25), 0.26884138359016674), ((0, 26), 0.3616982138868891), ((0, 27), 0.03902982752262633), ((0, 28), 0.16447171309457276), ((0, 29), 0.05398367288952082), ((0, 30),

True